In [1]:
!pip install spark-nlp==1.7.3

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StringType, DataType
import pandas as pd
from pyspark.sql.functions import udf,col
from pyspark.sql import functions as F
pd.set_option('max_colwidth', 800)
import matplotlib
from pyspark.sql.types import TimestampType, StructType, StructField, StringType, FloatType, IntegerType, LongType
from pyspark.sql import types as T
from datetime import datetime

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName(" Sparkify Localy") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2") \
    .getOrCreate()
spark

In [4]:
# get parameters
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1584626360751'),
 ('spark.app.name', ' Sparkify Localy'),
 ('spark.jars.packages', 'JohnSnowLabs:spark-nlp:1.8.2'),
 ('spark.repl.local.jars',
  'file:///home/anthelix/.ivy2/jars/JohnSnowLabs_spark-nlp-1.8.2.jar,file:///home/anthelix/.ivy2/jars/com.typesafe_config-1.3.0.jar,file:///home/anthelix/.ivy2/jars/org.rocksdb_rocksdbjni-5.17.2.jar,file:///home/anthelix/.ivy2/jars/org.apache.hadoop_hadoop-aws-2.7.3.jar,file:///home/anthelix/.ivy2/jars/com.amazonaws_aws-java-sdk-1.7.4.jar,file:///home/anthelix/.ivy2/jars/org.tensorflow_tensorflow-1.8.0.jar,file:///home/anthelix/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,file:///home/anthelix/.ivy2/jars/com.navigamez_greex-1.0.jar,file:///home/anthelix/.ivy2/jars/net.sf.trove4j_trove4j-3.0.3.jar,file:///home/anthelix/.ivy2/jars/commons-logging_commons-logging-1.1.3.jar,file:///home/anthelix/.ivy2/jars/org.apache.httpcomponents_httpclient-4.2.5.jar,file:///home/anthelix/.ivy2/jars/joda-time_joda-time

## Log Data Default Schema

In [5]:
log_data_path = "./log_data/*.json"
df_log = spark.read.json(log_data_path)
df_log
print('DataFrame rows: %d' % df_log.count())
df_log.printSchema()
print('DataFrame schema: %s' % df_log)
df_log.show(2)

DataFrame rows: 8056
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

DataFrame schema: DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint

## Log Data My Schema

In [8]:
log_data_path = "./log_data/*.json"
log_schema = StructType([
    StructField("artist", StringType()),
    StructField("auth", StringType()),
    StructField("firstName", StringType()),
    StructField("gender", StringType()),
    StructField("itemInSession", IntegerType()),
    StructField("lastName", StringType()),
    StructField("length", FloatType()),    
    StructField("level", StringType()),
    StructField("location", StringType()),
    StructField("method", StringType()),
    StructField("page", StringType()),
    StructField("registration", FloatType()),
    StructField("sessionId", StringType()),
    StructField("song", StringType()),
    StructField("status", IntegerType()),
    StructField("ts", LongType()),
    StructField("userAgent", StringType()),
    StructField("userId", StringType())
])
df_logS = spark.read.json(log_data_path, schema=log_schema)
df_logS
print('DataFrame rows: %d' % df_logS.count())
df_logS.printSchema()
print('DataFrame schema: %s' % df_logS)
df_logS.show(2)

DataFrame rows: 8056
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: integer (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: float (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: float (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

DataFrame schema: DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: int, lastName: string, length: float, level: string, location: string, method: string, page: string, registration: float, sessionId: string, song: string, status: int, 

## Song Data My schema

In [7]:
song_data_path = "./song_data/A/A/A/*.json"
song_schema = StructType([
    StructField("num_songs", IntegerType()),
    StructField("artist_id", StringType()),
    StructField("artist_latitude", FloatType()),
    StructField("artist_longitude", FloatType()),
    StructField("artist_location", StringType()),
    StructField("artist_name", StringType()),
    StructField("song_id", StringType()),
    StructField("title", StringType()),
    StructField("duration", FloatType()),
    StructField("year", IntegerType())   
])
df_song = spark.read.json(song_data_path, schema = song_schema)
print('DataFrame rows: %d' % df_song.count())
df_song.printSchema()
print('DataFrame schema: %s' % df_song)
df_song.show(3)

DataFrame rows: 11
root
 |-- num_songs: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: float (nullable = true)
 |-- artist_longitude: float (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- year: integer (nullable = true)

DataFrame schema: DataFrame[num_songs: int, artist_id: string, artist_latitude: float, artist_longitude: float, artist_location: string, artist_name: string, song_id: string, title: string, duration: float, year: int]
+---------+------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+---------+----+
|num_songs|         artist_id|artist_latitude|artist_longitude|     artist_location|         artist_name|           song_id|               title| duration|year|
+---------+------